In [18]:
import pandas as pd
df  =pd.read_csv('../Data/csv/version1.csv').astype(str)
df = df.drop("Unnamed: 0",axis=1)
df

,label,content
0,0,﻿ 第 章全 不知 幻想 一个 童话国 里 住 男孩儿 女孩儿 丁点儿 男孩儿 女孩儿 叫做...
1,0,﻿ 第一章 万事通 战胜 小星星 教授 全 不知 太阳城 旅行 回来 已经 过去 两年 半 ...
2,0,﻿ 第 章 花城 矮子 一座 童话 城市 里 住 矮子 管 叫 矮子 是因为 很小 每个 矮...
3,0,一个 贵族 女儿 风儿 草 吹 过去 时候 田野 就 像 湖水 一起 涟漪 麦子 扫...
4,0,一朵 叫 童话 蒲公英 斑马 一起 奔跑 一个 懒洋洋 季节 茉莉 正 坐在 树 发呆 一匹...
...,...,...
651,1,第 章鸡王 西双版纳 盛行 斗鸡 逢年过节 村村寨寨 都 举行 斗鸡 会 热闹 要 ...
652,1,麻瓜 魔女 永恒 天使 第 章 序幕 天使 之城 老 天使 拉索 魔法 独 坐在 隐...
653,1,一个 清澈 透明 冰晶 建造 宫殿 中 一个 蓝色 侏儒 坐在 冰雪 宝座 之上 殿...
654,1,第 章 母熊 大白 掌 老 猎人 亢浪隆 山林 里 闯荡 几十年 飞禽走兽 大半辈子...


In [8]:
import numpy as np
a = np.array([1,3,4])

a

In [9]:
a[:,None]

array([[1],
       [3],
       [4]])

In [11]:
a

array([[1],
       [3],
       [4]])

In [19]:
from sklearn.model_selection import train_test_split
import pandas as pd
import jieba
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['content'])
vocab=tokenizer.word_index #得到每个词的编号
x_train, x_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.2)
x_train_word_ids=tokenizer.texts_to_sequences(x_train)
x_test_word_ids = tokenizer.texts_to_sequences(x_test)

读取词向量模型，对所有词语向量化，如果在模型中没出现过，都按零向量处理

In [20]:
from gensim.models import keyedvectors
import gensim
import numpy as np
#w2v=keyedvectors.load_word2vec_format(os.path.join(root_path,"wiki_word2vec_50.bin"),binary=True)
w2v_model = gensim.models.KeyedVectors.load_word2vec_format("../Data/word2vec/wiki_word2vec_50.bin",binary=True)
embedding_matrix = np.zeros((len(vocab) + 1, 50))
for word, i in vocab.items():
    try:
        embedding_vector = w2v_model[str(word)]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        continue

In [28]:
from keras.layers.merge import concatenate
from keras.models import Model
from keras import  Input
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Embedding,Conv1D,MaxPooling1D,Flatten,Dense,Dropout,BatchNormalization
import sklearn.metrics as metrics
def TextCNN_model_2(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test,embedding_matrix,shape1):
    # 模型结构：词嵌入-卷积池化*3-拼接-全连接-dropout-全连接
    main_input = Input(shape=(shape1,), dtype='float64')
    # 词嵌入（使用预训练的词向量）
    embedder = Embedding(len(vocab) + 1, 50, input_length=shape1, weights=[embedding_matrix], trainable=False)
    embed = embedder(main_input)
    # 词窗大小分别为3,4,5
    cnn1 = Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)
    cnn1 = MaxPooling1D(pool_size=38)(cnn1)
    cnn2 = Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)
    cnn2 = MaxPooling1D(pool_size=37)(cnn2)
    cnn3 = Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)
    cnn3 = MaxPooling1D(pool_size=36)(cnn3)
    # 合并三个模型的输出向量
    cnn = concatenate([cnn1, cnn2, cnn3], axis=1)
    flat = Flatten()(cnn)
    drop = Dropout(0.4)(flat)
    main_output = Dense(2, activation='softmax')(drop)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    one_hot_labels = np_utils.to_categorical(y_train, num_classes=2)  # 将标签转换为one-hot编码
    model.fit(x_train_padded_seqs, one_hot_labels, batch_size=32, epochs=25)
    #y_test_onehot = keras.utils.to_categorical(y_test, num_classes=3)  # 将标签转换为one-hot编码
    result = model.predict(x_test_padded_seqs)  # 预测样本属于每个类别的概率
    result_labels = np.argmax(result, axis=1)  # 获得最大概率对应的标签
    y_predict = list(map(str, result_labels))
    y_pred = list(result_labels)
    t_test = list(y_test)
    #print(y_pred)
    print('准确率', metrics.accuracy_score(y_test, y_predict))
    print('平均f1-score:', metrics.f1_score(y_test, y_predict, average='weighted'))
#TextCNN_model_2(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test,embedding_matrix,)

In [33]:
x_train_padded_seqs=pad_sequences(x_train_word_ids,maxlen=500) #将超过固定值的部分截掉，不足的在最前面用0填充
x_test_padded_seqs=pad_sequences(x_test_word_ids, maxlen=500)
TextCNN_model_2(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test,embedding_matrix,500)

Epoch 1/25
17/17 [==============================] - 0s 16ms/step - loss: 1.3049 - accuracy: 0.5420
Epoch 2/25
17/17 [==============================] - 0s 13ms/step - loss: 0.6562 - accuracy: 0.6603
Epoch 3/25
17/17 [==============================] - 0s 13ms/step - loss: 0.4830 - accuracy: 0.7691
Epoch 4/25
17/17 [==============================] - 0s 13ms/step - loss: 0.4131 - accuracy: 0.8111
Epoch 5/25
17/17 [==============================] - 0s 13ms/step - loss: 0.3293 - accuracy: 0.8950
Epoch 6/25
17/17 [==============================] - 0s 13ms/step - loss: 0.3033 - accuracy: 0.8912
Epoch 7/25
17/17 [==============================] - 0s 13ms/step - loss: 0.2571 - accuracy: 0.9313
Epoch 8/25
17/17 [==============================] - 0s 13ms/step - loss: 0.2157 - accuracy: 0.9427
Epoch 9/25
17/17 [==============================] - 0s 13ms/step - loss: 0.1975 - accuracy: 0.9466
Epoch 10/25
17/17 [==============================] - 0s 13ms/step - loss: 0.1873 - accuracy: 0.9504
Epoch 11/

In [30]:
print(y_test)

62     0
165    0
574    1
307    0
586    1
      ..
26     0
187    0
166    0
226    0
112    0
Name: label, Length: 132, dtype: object


In [26]:
y_pred

NameError: name 'y_pred' is not defined

从50词到10000词训练查看结果

In [ ]:
p = 50
for i in range(200):
    x_train_padded_seqs=pad_sequences(x_train_word_ids,maxlen=p) #将超过固定值的部分截掉，不足的在最前面用0填充
    x_test_padded_seqs=pad_sequences(x_test_word_ids, maxlen=p)
    print("词汇量",p)
    TextCNN_model_2(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test,embedding_matrix,p)
    p+=50